###  p_1 != 0

### 1. calculate D(e,n)

In [1]:
from random import choices
import numpy as np

In [2]:
def unary_binary_subtrees(e, n):
    if e==0:
        return 0
    elif n==0:
        return 1
    else:
        return unary_binary_subtrees(e-1, n) + unary_binary_subtrees(e, n-1) + unary_binary_subtrees(e+1, n-1)

### 2. calculate distribution K(e,n)

In [3]:
def distribution_k_a(e, n):
    population = []
    weights = []
    for k in range(e):
        population.append( (k, 1))
        weights.append(unary_binary_subtrees(e-k, n-1)/unary_binary_subtrees(e, n))
        population.append((k, 2))
        weights.append(unary_binary_subtrees(e-k+1, n-1)/unary_binary_subtrees(e, n))
    return population, weights

### 3. leaves and binary operators

In [4]:
leaves = ["x", "-5", "-4", "-3", "-2", "-1", "1", "2", "3", "4", "5"]
binary_operators = ["+", "-", "*", "/"]
unary_operators = ["exp", "log", "sqrt", "sin", "cos", "tan", "arcsin", "arccos", "arctan", "sinh", "cosh", "tanh", "sinh", "cosh", "tanh"]

### 4. generate random binary trees

In [5]:
class Node_Binary:
    operator = True
    binary = True
    operand = False
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right
        
class Node_Unary:
    operator = True
    binary = False
    operand = False
    def __init__(self, data, middle=None):
        self.data = data
        self.middle = middle 
        
class Leaf:
    operator = False
    binary = False
    operand = True
    def __init__(self, data):
        self.data = data

In [10]:
def random_binary_trees(n):
    population, weights = distribution_k_a(1, n)
    k, a = choices(population, weights)[0]
    if a == 1:
        operator = choices(unary_operators)[0]
        node = Node_Unary(operator)
        empty_node = [(node, 'middle')]
        e = 1
    else:
        operator = choices(binary_operators)[0]
        node = Node_Binary(operator)
        empty_node = [(node, 'left'), (node, 'right')]
        e = 2
    n = n - 1
    while n > 0:
        population, weights = distribution_k_a(e, n)
        k, a = choices(population, weights)[0]
        i = 0
        new_empty_node = []
        for ele in empty_node:
            if i < k:
                leave = choices(leaves)[0]
                value = Leaf(leave)
                setattr(ele[0], ele[1], value)
            elif i == k:
                if a == 1:
                    operator = choices(unary_operators)[0]
                    value = Node_Unary(operator)
                    setattr(ele[0], ele[1], value)
                    if ele[1] == 'left':
                        new_ele = ele[0].left
                    elif ele[1] == 'middle':
                        new_ele = ele[0].middle
                    else:
                        new_ele = ele[0].right
                    new_empty_node = [(new_ele, 'middle')]
                    e = e - k
                else:
                    operator = choices(binary_operators)[0]
                    value = Node_Binary(operator)
                    setattr(ele[0], ele[1], value)
                    if ele[1] == 'left':
                        new_ele = ele[0].left
                    elif ele[1] == 'middle':
                        new_ele = ele[0].middle
                    else:
                        new_ele = ele[0].right
                    new_empty_node = [(new_ele, 'left'), (new_ele, 'right')]
                    e = e - k + 1
            else:
                new_empty_node.append(ele)
            i += 1
        empty_node = new_empty_node
        n = n - 1
    if len(empty_node) != 0:
        for ele in empty_node:
            leave = choices(leaves)[0]
            value = Leaf(leave)
            setattr(ele[0], ele[1], value)
            
    return node

In [18]:
root = random_binary_trees(4)

In [19]:
def traverse_unary_binary(root, seq=None, verbose=False):
    if not seq:
        seq = []
        
    if verbose:
        print(root.data)
        
    seq.append(root.data)
    
    if root.binary:
        if root.left.operator:
            traverse_unary_binary(root.left, seq)
        else:
            if verbose:
                print(root.left.data)
            seq.append(root.left.data)
        
        if root.right.operator:
            traverse_unary_binary(root.right, seq)
        else:
            if verbose:
                print(root.right.data)
            seq.append(root.right.data)
    else:
        if root.middle.operator:
            traverse_unary_binary(root.middle, seq)
        else:
            if verbose:
                print(root.middle.data)
            seq.append(root.middle.data)
    
    return seq
        
traverse_unary_binary(root) 

['cosh', '-', '4', 'sinh', '/', '-4', '-2']

- Test traverse <br>
need to add test for unary-binary mixed expression

In [ ]:
def test_traverse():
    
    # prefix = (5 − 6) × 7
    tree = Node('*')
    tree.left = Node('-')
    tree.right = Leaf(7)
    tree.left.left = Leaf(5)
    tree.left.right = Leaf(6)
    
    res = traverse(tree, verbose=False)
    desired = ['*', '-', 5, 6, 7]
    assert desired == res
    
test_traverse()

In [ ]:
def test_traverse2():
    
    # prefix = 2 + 3 * (5 + 2)
    tree = Node('+')
    tree.left = Leaf(2)
    tree.right = Node('*')
    tree.right.left = Leaf(3)
    tree.right.right = Node('+')
    tree.right.right.left = Leaf(5)
    tree.right.right.right = Leaf(2)

    res = traverse(tree, verbose=False)
    desired = ['+', 2, '*', 3, '+', 5, 2]
    assert desired == res
    
test_traverse2()

<pre>
Algorithm infix (tree)
 if (tree not empty)
    if (tree token is operator)
       print (open parenthesis)
    end if
    infix (tree left subtree)
    print (tree token)
    infix (tree right subtree)
    if (tree token is operator)
       print (close parenthesis)
    end if
 end if
end infix
<pre>

In [26]:
def traverse_unary_binary_infix(root, seq=None):
    if not seq:
        seq = []
    if root.operator:
        if root.binary:
            seq.append("(")    
            if root.left.operator:
                traverse_unary_binary_infix(root.left, seq)
            else:
                seq.append(root.left.data)
            seq.append(root.data)    
            if root.right.operator:
                traverse_unary_binary_infix(root.right, seq)
            else:
                seq.append(root.right.data)    
            seq.append(")")
        else:
            seq.append(root.data)
            seq.append("(")
            if root.middle.operator:
                traverse_unary_binary_infix(root.middle, seq)
            else:
                seq.append(root.middle.data)  
            seq.append(")") 
    return seq       
seq = traverse_unary_binary_infix(root)                

In [29]:
print(seq)

['cosh', '(', '(', '4', '-', 'sinh', '(', '(', '-4', '/', '-2', ')', ')', ')', ')']


<pre>
Algorithm prefix (tree) 
 if (tree not empty) 
    print (tree token) 
    prefix (tree left subtree) 
    prefix (tree right subtree)
 end if 
end prefix <br>
<pre>

In [28]:
def traverse_unary_binary_prefix(root, seq=None, verbose=False):
    if not seq:
        seq = []
        
    if verbose:
        print(root.data)
        
    seq.append(root.data)
    
    if root.binary:
        if root.left.operator:
            traverse_unary_binary(root.left, seq)
        else:
            if verbose:
                print(root.left.data)
            seq.append(root.left.data)
        
        if root.right.operator:
            traverse_unary_binary(root.right, seq)
        else:
            if verbose:
                print(root.right.data)
            seq.append(root.right.data)
    else:
        if root.middle.operator:
            traverse_unary_binary(root.middle, seq)
        else:
            if verbose:
                print(root.middle.data)
            seq.append(root.middle.data)
    
    return seq
        
traverse_unary_binary_prefix(root)  

['cosh', '-', '4', 'sinh', '/', '-4', '-2']